In [113]:
import numpy as np

In [114]:
# シードを固定
np.random.seed(45556)

In [115]:
# 入力データ
S = 3  # リソースの数
N = 4  # プレイヤーの数

In [116]:
# リソースの総数 q_s (ランダムな整数で生成)
q = np.random.randint(5, 15, size=S)
print(q)

[12 11  6]


In [117]:
# 各リソースの参入障壁 c_s (ランダムな整数で生成)
c = np.random.randint(1, 5, size=S)
print(c)


[4 3 2]


In [118]:
# プレイヤーごとの総予算 b_i (ランダムな整数で生成)
b = np.random.randint(10, 20, size=N)
print(b)


[10 10 14 19]


In [119]:
# プレイヤーの各リソースに対する入札 x_{is} (合計が予算と一致するようにランダム生成)
x = np.zeros((N, S))
for i in range(N):
    remaining_budget = b[i]
    for s in range(S-1):
        x[i, s] = np.random.randint(0, remaining_budget)
        remaining_budget -= x[i, s]
    x[i, S-1] = remaining_budget
print(x)

[[ 9.  0.  1.]
 [ 1.  0.  9.]
 [11.  2.  1.]
 [ 4.  8.  7.]]


In [120]:

# 各プレイヤーのマージナルゲイン g_i (ランダムな浮動小数点数で生成)
g = np.random.uniform(1.0, 2.0, size=N)
print(g)

[1.99634994 1.0652844  1.41243639 1.79564471]


In [121]:
# リソースの割り当て計算
rho = np.zeros((N, S))
for i in range(N):
    for s in range(S):
        rho[i, s] = (q[s] * x[i, s]) / (c[s] + np.sum(x[:, s]))

print(rho)

[[3.72413793 0.         0.3       ]
 [0.4137931  0.         2.7       ]
 [4.55172414 1.69230769 0.3       ]
 [1.65517241 6.76923077 2.1       ]]


In [122]:
# プレイヤーのユーティリティ計算
utilities = np.zeros(N)
for i in range(N):
    for s in range(S):
        utilities[i] += g[i] * rho[i, s] - x[i, s]

print(utilities)

[-1.96641249 -6.68292477 -4.75697131 -0.10191112]


## monotone性の確認

In [123]:
def compute_gradient(x, q, c, g, N, S):
    grad = np.zeros((N, S))
    for i in range(N):
        for s in range(S):
            denominator = c[s] + np.sum(x[:, s])
            grad[i, s] = g[i] * (q[s] * c[s] + q[s] * np.sum(x[:, s]) - q[s] * x[i, s]) / (denominator ** 2) - 1
    return grad

In [124]:
# x'を新しい戦略プロファイルとして生成（ここではランダムに生成）
x_prime = np.zeros((N, S))
for i in range(N):
    remaining_budget = b[i]
    for s in range(S-1):
        x_prime[i, s] = np.random.randint(0, remaining_budget)
        remaining_budget -= x_prime[i, s]
    x_prime[i, S-1] = remaining_budget

In [125]:
# 利得勾配を計算
grad_x = compute_gradient(x, q, c, g, N, S)
grad_x_prime = compute_gradient(x_prime, q, c, g, N, S)

# 利得勾配の差を計算
grad_diff = grad_x_prime - grad_x

In [126]:
# 内積を計算
inner_product = 0
for i in range(N):
    for s in range(S):
        inner_product += grad_diff[i, s] * (x_prime[i, s] - x[i, s])

In [127]:
# 重み付き合計（ここではλ_iを1と仮定）
lambda_i = np.ones(N)
DSC = np.sum(lambda_i * inner_product)

print(DSC)

-44.63947538768198
